## Additional Preprocessing Steps for VQA-RAD, PathVQA, and SLAKE

In [1]:
import os
import os.path as osp
import pandas as pd
from PIL import Image
import re
import json
import pickle

import warnings
warnings.filterwarnings('ignore')

from llava.constants import DEFAULT_IMAGE_TOKEN

# Load LLaVA-Med alignment and instruction datasets for reference
llava_med_dir = '../../llava-med'

llava_med_align_dir = osp.join(llava_med_dir, 'data/alignment')
llava_med_align_data = json.load(open(osp.join(llava_med_align_dir, 'llava_med_alignment_500k.json')))

llava_med_instruct_dir = osp.join(llava_med_dir, 'data/instruct')
llava_med_instruct_data = json.load(open(osp.join(llava_med_instruct_dir, 'llava_med_instruct_10k.json')))

# VQA-RAD dataset
vqa_rad_dir = '/data/vqa-rad/'
vqa_rad_json = json.load(open(osp.join(vqa_rad_dir, 'VQA_RAD Dataset Public.json'))) # List of dictionaries
vqa_rad_df = pd.read_json(osp.join(vqa_rad_dir, 'VQA_RAD Dataset Public.json')) # Dataframe format
vqa_rad_df['answer'] = vqa_rad_df['answer'].apply(lambda x: str(x).lower().strip())
vqa_rad_df['answer_type'] = vqa_rad_df['answer_type'].apply(lambda x: str(x).lower().strip())
vqa_rad_image_dir = osp.join(vqa_rad_dir, 'images')
vqa_rad_images = os.listdir(vqa_rad_image_dir)

# PathVQA dataset
pvqa_dir = '/data/pvqa'
pvqa_image_dir = osp.join(pvqa_dir, 'images')
pvqa_train_images = [osp.join(pvqa_image_dir, f'train/{i}') for i in os.listdir(osp.join(pvqa_image_dir, 'train'))]
pvqa_val_images = [osp.join(pvqa_image_dir, f'val/{i}') for i in os.listdir(osp.join(pvqa_image_dir, 'val'))]
pvqa_test_images = [osp.join(pvqa_image_dir, f'test/{i}') for i in os.listdir(osp.join(pvqa_image_dir, 'test'))]

# SLAKE dataset
slake_dir = '/data/slake'
slake_image_dir = osp.join(slake_dir, 'imgs')

Format used by LLaVA-Med for medical concept alignment:

In [2]:
llava_med_align_data[0]

{'id': '32357477_ijms-21-03049-f006',
 'image': '32357477_ijms-21-03049-f006.jpg',
 'conversatons': [{'from': 'human',
   'value': 'Illustrate the image through a descriptive explanation\n<image>'},
  {'from': 'gpt',
   'value': 'Determination of metaphase II entry in oocytes pre-exposed to ferrocenyl 4-(alkylamino)-1,4-dihydroquinolines. After incubation or not with compounds 9, 6, 10, 7, 11, 8 for 24 h, oocytes were rinsed four times in ND96 for 30 min, before progesterone stimulation. White spot appearance was scored after 15 h. N refers to the number of females and n to the number of oocytes (N = 2 and n = 60).'}]}

Format used by LLaVA-Med for visual instruction tuning:

In [3]:
llava_med_instruct_data[0]

{'id': '17506892_F1',
 'image': '17506892_F1.jpg',
 'conversatons': [{'from': 'human',
   'value': 'Can you describe the image for me?\n<image>'},
  {'from': 'gpt',
   'value': 'The image consists of maps of significant voxels representing regions of hypoperfusion in FTLD patients according to their clinical diagnosis. These maps are superimposed onto a reference T1-weighted MRI image. There are five rows, each representing a different patient subgroup: bvFTD, SD, PNFA, PSP, and CBD. The neurological convention is followed, with the left side of the brain on the left side of the image.'},
  {'from': 'human', 'value': 'What is the modality of this image?'},
  {'from': 'gpt',
   'value': "The modality of this image is a T1-weighted MRI, which is a type of magnetic resonance imaging used to visualize the brain's anatomy."},
  {'from': 'human',
   'value': 'What stands out or draws attention in this image?'},
  {'from': 'gpt',
   'value': 'What stands out in this image are the regions of h

## VQA-RAD

In [2]:
vqa_rad_df.head()

,qid,phrase_type,qid_linked_id,image_case_url,image_name,image_organ,evaluation,question,question_rephrase,question_relation,question_frame,question_type,answer,answer_type
0,0,freeform,03f451ca-de62-4617-9679-e836026a7642,https://medpix.nlm.nih.gov/case?id=48e1dd0e-85...,synpic54610.jpg,HEAD,not evaluated,Are regions of the brain infarcted?,NULL,NULL,NULL,PRES,yes,closed
1,1,freeform,06e26b2c-04b9-42bc-8e98-1de30a0f7682,https://medpix.nlm.nih.gov/case?id=b197277b-69...,synpic29265.jpg,CHEST,not evaluated,Are the lungs normal appearing?,NULL,NULL,NULL,ABN,no,closed
2,2,freeform,0d0e8b6b-7753-4788-9b6d-dc7f25250c3f,https://medpix.nlm.nih.gov/case?id=b197277b-69...,synpic29265.jpg,CHEST,not evaluated,Is there evidence of a pneumothorax,NULL,NULL,NULL,PRES,no,closed
3,3,freeform,0e90b6bc-265f-490b-a039-509b9907a3cb,https://medpix.nlm.nih.gov/case?id=19aa8a2b-35...,synpic28602.jpg,CHEST,given,What type of imaging does this not represent?,NULL,NULL,NULL,MODALITY,ultrasound,open
4,4,freeform,1179f612-12e0-4dda-aee0-f14a5200be7b,https://medpix.nlm.nih.gov/case?id=b197277b-69...,synpic29265.jpg,CHEST,given,Is this a MRI of the chest?,NULL,NULL,NULL,MODALITY,no,closed


### Extracting the closed-ended QA pairs for evaluation

In [8]:
# Extract only closed-ended QA pairs
df = vqa_rad_df[(vqa_rad_df['answer_type'] == 'closed')]

# Reference: https://huggingface.co/datasets/flaviagiammarino/vqa-rad/blob/main/scripts/processing.py
# Retrieve the train and test splits
train_df = df[df['phrase_type'].isin(['freeform', 'para'])]
test_df = df[df['phrase_type'].isin(['test_freeform', 'test_para'])]

# Remove duplicate examples
subset_cols = ['image_name', 'question', 'answer']
train_df = train_df.drop_duplicates(subset=subset_cols, ignore_index=True)
test_df = test_df.drop_duplicates(subset=subset_cols, ignore_index=True)

# Remove data leakage
train_df = train_df[~train_df[subset_cols].apply(tuple, 1).isin(test_df[subset_cols].apply(tuple, 1))]

# Clean up QA formatting 
f = lambda x: re.sub(' +', ' ', str(x).lower()).replace(" ?", "?").strip()
train_df['question'] = train_df['question'].apply(f)
train_df['answer'] = train_df['answer'].apply(f)
test_df['question'] = test_df['question'].apply(f)
test_df['answer'] = test_df['answer'].apply(f)

# Reindex the examples
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# NOTE: For the closed-ended questions whose answer is not yes/no, we manually fix the options in the .jsonl files to follow what the question is asking.
def convert_to_llava_instruct_format(df):
    converted = []
    
    for i, row in df.iterrows():
        question = row['question'].capitalize()
        answer = row['answer'].lower().strip()

        # Build conversation
        row_dict = dict(
            id=osp.splitext(row['image_name'])[0],
            image=row['image_name'],
            conversations=[
                {'from': 'human', 'value': f'{DEFAULT_IMAGE_TOKEN}\n{question}'},
                {'from': 'gpt', 'value': answer}
            ],
            options=['yes', 'no']
        )
        converted.append(row_dict)

    return converted

# Converted to JSON format
# NOTE: Should have 272 QA pairs
train_json_orig = convert_to_llava_instruct_format(train_df)
test_json_orig = convert_to_llava_instruct_format(test_df)

closed_dir = osp.join(vqa_rad_dir, 'closed')
os.makedirs(closed_dir, exist_ok=True)

def write_jsonl(data, path):
    with open(path, 'w') as fh:
        for qa in data:
            json.dump(qa, fh)
            fh.write('\n')

def read_jsonl(path):
    data = []
    with open(path, 'r') as fh:
        for line in fh:
            data.append(json.loads(line))

    return data

write_jsonl(train_json_orig, osp.join(closed_dir, 'train_orig.jsonl'))
write_jsonl(test_json_orig, osp.join(closed_dir, 'test_orig.jsonl'))

In [31]:
# Sanity check
# Reference: https://github.com/haotian-liu/LLaVA/blob/main/docs/Finetune_Custom_Data.md
read_jsonl(osp.join(closed_dir, 'test_orig.jsonl'))[0]

{'id': 'synpic42202',
 'image': 'synpic42202.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nIs there evidence of an aortic aneurysm?'},
  {'from': 'gpt', 'value': 'yes'}],
 'options': ['yes', 'no']}

In [22]:
# Compare with LLaVA-Med instruction data
llava_med_instruct_data[3]

{'id': '18021452_F4',
 'image': '18021452_F4.jpg',
 'conversatons': [{'from': 'human',
   'value': '<image>\nWhat does the image show in a few words?'},
  {'from': 'gpt',
   'value': 'The image shows a carcinoid tumor penetrating the subserosa of the small bowel, with H&E staining at 20x magnification.'},
  {'from': 'human', 'value': 'What is the modality of this image?'},
  {'from': 'gpt',
   'value': 'The modality of this image is a light microscopy with Hematoxylin and Eosin (H&E) staining.'},
  {'from': 'human',
   'value': 'What are the key or essential components of this image?'},
  {'from': 'gpt',
   'value': 'The key components of this image are the small bowel tissue, the carcinoid tumor, the subserosa layer, and the H&E staining that highlights the cellular structures.'},
  {'from': 'human',
   'value': 'Can you describe the appearance of the carcinoid tumor in the image?'},
  {'from': 'gpt',
   'value': 'The carcinoid tumor appears as an irregular mass with distinct cellular

## PathVQA

In [22]:
# Load the QA files
pvqa_train_pkl = pickle.load(open(osp.join(pvqa_dir, 'qas/train/train_qa.pkl'), 'rb'))
pvqa_val_pkl = pickle.load(open(osp.join(pvqa_dir, 'qas/val/val_qa.pkl'), 'rb'))
pvqa_test_pkl = pickle.load(open(osp.join(pvqa_dir, 'qas/test/test_qa.pkl'), 'rb'))

# Convert to DataFrame
pvqa_train_df = pd.DataFrame.from_records(pvqa_train_pkl)
pvqa_val_df = pd.DataFrame.from_records(pvqa_val_pkl)
pvqa_test_df = pd.DataFrame.from_records(pvqa_test_pkl)

# Add file format extension to image column
pvqa_train_df['image'] = pvqa_train_df['image'].apply(lambda x: f'{x}.jpg')
pvqa_val_df['image'] = pvqa_val_df['image'].apply(lambda x: f'{x}.jpg')
pvqa_test_df['image'] = pvqa_test_df['image'].apply(lambda x: f'{x}.jpg')

# Add new column indicating whether the question is closed- or open-ended
# NOTE: The original PathVQA dataset 
pvqa_train_df['answer_type'] = pvqa_train_df['answer'].apply(
    lambda x: 'closed' if (
        len(re.findall(r'\byes\b|\bno\b', x)) > 0 and x.lower().strip() in ['yes', 'no']
    ) else 'open'
)
pvqa_val_df['answer_type'] = pvqa_val_df['answer'].apply(
    #lambda x: 'closed' if ('yes' in x.lower() or 'no' in x.lower()) else 'open'
    lambda x: 'closed' if (
        len(re.findall(r'\byes\b|\bno\b', x)) > 0 and x.lower().strip() in ['yes', 'no']
    ) else 'open'
)
pvqa_test_df['answer_type'] = pvqa_test_df['answer'].apply(
    #lambda x: 'closed' if ('yes' in x.lower() or 'no' in x.lower()) else 'open'
    lambda x: 'closed' if (
        len(re.findall(r'\byes\b|\bno\b', x)) > 0 and x.lower().strip() in ['yes', 'no']
    ) else 'open'
)

print('Total number of QA pairs: {}'.format(
    pvqa_train_df.shape[0] + pvqa_val_df.shape[0] + pvqa_test_df.shape[0]
))
print('Total number of closed-ended yes/no questions: {}'.format(
    pvqa_train_df[pvqa_train_df['answer_type'] == 'closed'].shape[0] + \
    pvqa_val_df[pvqa_val_df['answer_type'] == 'closed'].shape[0] + \
    pvqa_test_df[pvqa_test_df['answer_type'] == 'closed'].shape[0]
))
print('Yes: {}, No: {}'.format(
    pvqa_train_df[pvqa_train_df['answer'] == 'yes'].shape[0] + \
    pvqa_val_df[pvqa_val_df['answer'] == 'yes'].shape[0] + \
    pvqa_test_df[pvqa_test_df['answer'] == 'yes'].shape[0],
    pvqa_train_df[pvqa_train_df['answer'] == 'no'].shape[0] + \
    pvqa_val_df[pvqa_val_df['answer'] == 'no'].shape[0] + \
    pvqa_test_df[pvqa_test_df['answer'] == 'no'].shape[0]
))

Total number of QA pairs: 32795
Total number of closed-ended yes/no questions: 16332
Yes: 8993, No: 7339


Since we're only interested in evaluating on closed-ended questions, we filter out all closed-ended questions.

In [25]:
pvqa_train_df = pvqa_train_df[pvqa_train_df['answer_type'] == 'closed'].reset_index(drop=True)
pvqa_val_df = pvqa_val_df[pvqa_val_df['answer_type'] == 'closed'].reset_index(drop=True)
pvqa_test_df = pvqa_test_df[pvqa_test_df['answer_type'] == 'closed'].reset_index(drop=True)

Here, we convert each `.pkl` file into a `.json` file with the same format as that of the alignment and VQA-RAD datasets.

In [27]:
def convert_to_llava_instruct_format(df, q_col='question', a_col='answer', img_col='image_name'):
    converted = []
    
    for i, row in df.iterrows():
        question = row[q_col].capitalize()
        answer = row[a_col].lower().strip()

        # Build conversation
        row_dict = dict(
            id=osp.splitext(row[img_col])[0],
            image=row[img_col],
            conversations=[
                {'from': 'human', 'value': f'{DEFAULT_IMAGE_TOKEN}\n{question}'},
                {'from': 'gpt', 'value': answer}
            ],
            options=['yes', 'no']
        )
        converted.append(row_dict)

    return converted

pvqa_train_final = convert_to_llava_instruct_format(pvqa_train_df, img_col='image')
pvqa_val_final = convert_to_llava_instruct_format(pvqa_val_df, img_col='image')
pvqa_test_final = convert_to_llava_instruct_format(pvqa_test_df, img_col='image')

pvqa_closed_dir = osp.join(pvqa_dir, 'closed')
os.makedirs(pvqa_closed_dir, exist_ok=True)

def write_jsonl(data, path):
    with open(path, 'w') as fh:
        for qa in data:
            json.dump(qa, fh)
            fh.write('\n')

def read_jsonl(path):
    data = []
    with open(path, 'r') as fh:
        for line in fh:
            data.append(json.loads(line))

    return data

write_jsonl(pvqa_train_final, osp.join(pvqa_closed_dir, 'train.jsonl'))
write_jsonl(pvqa_val_final, osp.join(pvqa_closed_dir, 'val.jsonl'))
write_jsonl(pvqa_test_final, osp.join(pvqa_closed_dir, 'test.jsonl'))

In [28]:
# Sanity check
read_jsonl(osp.join(pvqa_closed_dir, 'test.jsonl'))[0]

{'id': 'test_0167',
 'image': 'test_0167.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nAre the histone subunits positively charged,  thus allowing the compaction of the negatively charged dna?'},
  {'from': 'gpt', 'value': 'yes'}],
 'options': ['yes', 'no']}

In [34]:
llava_med_instruct_data[0]

{'id': '17506892_F1',
 'image': '17506892_F1.jpg',
 'conversatons': [{'from': 'human',
   'value': 'Can you describe the image for me?\n<image>'},
  {'from': 'gpt',
   'value': 'The image consists of maps of significant voxels representing regions of hypoperfusion in FTLD patients according to their clinical diagnosis. These maps are superimposed onto a reference T1-weighted MRI image. There are five rows, each representing a different patient subgroup: bvFTD, SD, PNFA, PSP, and CBD. The neurological convention is followed, with the left side of the brain on the left side of the image.'},
  {'from': 'human', 'value': 'What is the modality of this image?'},
  {'from': 'gpt',
   'value': "The modality of this image is a T1-weighted MRI, which is a type of magnetic resonance imaging used to visualize the brain's anatomy."},
  {'from': 'human',
   'value': 'What stands out or draws attention in this image?'},
  {'from': 'gpt',
   'value': 'What stands out in this image are the regions of h

## SLAKE

In [19]:
slake_train_df = pd.read_json(osp.join(slake_dir, 'train.json'))
slake_val_df = pd.read_json(osp.join(slake_dir, 'validate.json'))
slake_test_df = pd.read_json(osp.join(slake_dir, 'test.json'))

# Only take the QA pairs in English and closed-ended questions
slake_train_df = slake_train_df[(slake_train_df['q_lang'] == 'en') & (slake_train_df['answer_type'] == 'CLOSED')].reset_index(drop=True)
slake_val_df = slake_val_df[(slake_val_df['q_lang'] == 'en') & (slake_val_df['answer_type'] == 'CLOSED')].reset_index(drop=True)
slake_test_df = slake_test_df[(slake_test_df['q_lang'] == 'en') & (slake_test_df['answer_type'] == 'CLOSED')].reset_index(drop=True)

slake_train_df['answer_type'] = slake_train_df['answer_type'].apply(lambda x: x.lower())
slake_val_df['answer_type'] = slake_val_df['answer_type'].apply(lambda x: x.lower())
slake_test_df['answer_type'] = slake_test_df['answer_type'].apply(lambda x: x.lower())

print('Aftering only selecting closed-ended QA pairs in English:')
print(f'Train: {slake_train_df.shape[0]}')
print(f'Val: {slake_val_df.shape[0]}')
print(f'Test: {slake_test_df.shape[0]}')
print(f'Total: {slake_train_df.shape[0] + slake_val_df.shape[0] + slake_test_df.shape[0]}')

Aftering only selecting closed-ended QA pairs in English:
Train: 1943
Val: 422
Test: 416
Total: 2781


In [20]:
slake_train_df.head()

,img_id,img_name,question,answer,q_lang,location,modality,answer_type,base_type,content_type,triple,qid
0,1,xmlab1/source.jpg,Does the picture contain liver?,Yes,en,Abdomen,MRI,closed,vqa,Organ,"[vhead, _, _]",3
1,1,xmlab1/source.jpg,Does the picture contain kidney?,No,en,Abdomen,MRI,closed,vqa,Organ,"[vhead, _, _]",4
2,1,xmlab1/source.jpg,Does the picture contain spleen?,No,en,Abdomen,MRI,closed,vqa,Organ,"[vhead, _, _]",5
3,100,xmlab100/source.jpg,Does the picture contain liver?,Yes,en,Lung,CT,closed,vqa,Organ,"[vhead, _, _]",11
4,100,xmlab100/source.jpg,Does the picture contain lung?,Yes,en,Lung,CT,closed,vqa,Organ,"[vhead, _, _]",12


In [ ]:
def convert_to_llava_instruct_format(df, q_col='question', a_col='answer', img_col='img_name'):
    converted = []
    
    for i, row in df.iterrows():
        question = row[q_col].capitalize()
        answer = row[a_col].lower().strip()
        
        # Parse out the options
        if answer in ['yes', 'no']:
            options = ['yes', 'no']
        else:
            if 'Is brain tumor white or gray relative to other tissues?' in question:
                options = ['white', 'gray']
            
            elif 'Is the abnormality hyperdense or hypodense?' in question:
                options = ['hyperdense', 'hypodense']

            elif 'Is the brain enhancing tumor hyperdense or hypodense?' in question:
                options = ['hyperdense', 'hypodense']

            elif 'Is the brain non-enhancing tumor hyperdense or hypodense?' in question:
                options = ['hyperdense', 'hypodense']

            elif 'Is this a t1 weighted or t2 weighted mri image?' in question:
                options = ['t1', 't2']

            elif 'Which plane is the image scanned, transverse plane or coronal plane?' in question:
                options = ['transverse plane', 'coronal plane']
            
            elif 'Which is bigger in this image,left lung or left kidney?' in question:
                options = ['left lung', 'left kidney']

            elif 'Which is bigger in this image,small bowel or kidney?' in question:
                options = ['small bowel', 'kidney']

            elif 'Which is smaller in this image,liver or spinal cord?' in question:
                options = ['liver', 'spinal cord']

            elif 'Which is smaller in this image,liver or right kidney?' in question:
                options = ['liver', 'right kidney']

            elif 'Which is smaller in this image,kidney or spinal cord?' in question:
                options = ['kidney', 'spinal cord']

            elif 'Which is smaller in this image, small bowel or right kidney?' in question:
                options = ['small bowel', 'right kidney']

            elif 'Which is the smallest in this image,spleen,left kidney or liver?' in question:
                options = ['left kidney', 'liver']

            elif 'Which is the smallest in this image, colon, left lung or liver?' in question:
                options = ['colon', 'left lung', 'liver']

            elif 'Which is bigger in this image, heart or right lung?' in question:
                options = ['heart', 'right lung']

            elif 'Which is smaller in this image, bladder or small bowel?' in question:
                options = ['bladder', 'small bowel']

            elif 'Which is bigger in this image, colon or small bowel?' in question:
                options = ['colon', 'small bowel']

            elif 'Which is bigger, left kidney or spleen ?' in question:
                options = ['left kidney', 'spleen']

            elif 'Where is the spleen in this image, right or lower right?' in question:
                options = ['right', 'lower right']

            elif 'Which is smaller in this image,liver or left lung?' in question:
                options = ['liver', 'left lung']

            elif 'Which is bigger in this image, small bowel or kidney?' in question:
                options = ['small bowel', 'kidney']

            elif 'Where is the left kidney in this image, right or lower right?' in question:
                options = ['right', 'lower right']

            elif 'Which is bigger in this image,small bowel or liver?' in question:
                options = ['small bowel', 'liver']

            elif 'Which is smaller in this image,spleen or right kidney?' in question:
                options = ['spleen', 'right kidney']

            elif 'Which is bigger in this image, rectum or small bowel?' in question:
                options = ['rectum', 'small bowel']

            elif 'Which is bigger in this image, kidney or spleen?' in question:
                options = ['kidney', 'spleen']

            elif 'Which is bigger in this image, kidney or spleen?' in question:
                options = ['kidney', 'spleen']

            elif 'Which is bigger in this image, small bowel or colon?' in question:
                options = ['small bowel', 'colon']

            else:
                try:
                    option_str = question.split(',', 1)[1]
                    option_str = option_str.strip('?')
                    option_str = option_str.replace(',', ' ')
                    option_str = option_str.replace('or', ' ')
                    option_str = option_str.replace('  ', ' ')
                    options = [o.lower() for o in option_str.split(' ')]
                    options = [o for o in options if o != '']
                except:
                    options = []

        if len(options) == 0:
            print('No options found:')
            print(f'Question: {question}')
            print(f'Answer: {answer}\n')
            continue

        if len(options) > 0 and answer not in options:
            if answer in ['both', 'almost the same']:
                options.append(answer)
            else:
                print('Mismatch between question and answer; excluded:')
                print(f'Question: {question}')
                print(f'Options: {options}')
                print(f'Answer: {answer}\n')
                continue

        # Build conversation
        row_dict = dict(
            id=osp.splitext(row[img_col])[0],
            image=row[img_col],
            conversations=[
                {'from': 'human', 'value': f'{DEFAULT_IMAGE_TOKEN}\n{question}'},
                {'from': 'gpt', 'value': answer}
            ],
            options=options
        )
        converted.append(row_dict)

    return converted

slake_train_final = convert_to_llava_instruct_format(slake_train_df, img_col='img_name')
slake_val_final = convert_to_llava_instruct_format(slake_val_df, img_col='img_name')
slake_test_final = convert_to_llava_instruct_format(slake_test_df, img_col='img_name')

slake_closed_dir = osp.join(slake_dir, 'closed')
os.makedirs(slake_closed_dir, exist_ok=True)

def write_jsonl(data, path):
    with open(path, 'w') as fh:
        for qa in data:
            json.dump(qa, fh)
            fh.write('\n')

def read_jsonl(path):
    data = []
    with open(path, 'r') as fh:
        for line in fh:
            data.append(json.loads(line))

    return data

write_jsonl(slake_train_final, osp.join(slake_closed_dir, 'train.jsonl'))
write_jsonl(slake_val_final, osp.join(slake_closed_dir, 'val.jsonl'))
write_jsonl(slake_test_final, osp.join(slake_closed_dir, 'test.jsonl'))